# LAMsec Demo Notebook
Quick interactive demonstration of the defended Large Action Model (LAM) pipeline: generation, defenses, simulation, and metrics (ADS, SVI, GCR).

## 1. Environment Setup
If running on Google Colab uncomment the first cell to install dependencies.

In [ ]:
# Colab only: install (commented by default)
# !pip install -q transformers sentence-transformers rapidfuzz gradio pyyaml torch==2.2.0 --extra-index-url https://download.pytorch.org/whl/cpu
# (Optional) if repo not present: !git clone https://github.com/your-org/LAMsec.git && cd LAMsec

## 2. Imports & Seeding

In [ ]:
from pathlib import Path
import json, yaml
from src.model_wrapper import generate_and_parse
from src.defense import InputSanitizer, SemanticAnalyzer, ActionValidator, SafeAlternativeGenerator, policy_decide
from src.metrics import ads, svi, gcr, attack_success
from src.envs.text_nav import TextNavigationEnv
from src.utils import set_deterministic
set_deterministic(42)
print('Imports loaded.')

## 3. Load Attack Dataset & Select a Sample

In [ ]:
data_path = Path('attacks/attacks.yaml')
attacks = yaml.safe_load(data_path.read_text())
len(attacks), attacks[0]['id']

## 4. Define a helper to run pipeline for one (intended, injection) pair

In [ ]:
sanitizer = InputSanitizer()
semantic = SemanticAnalyzer()
validator = ActionValidator()
safe_gen = SafeAlternativeGenerator()

def run_case(entry):
    intended = entry['intended']
    injected = entry['injection']
    intended_goal = entry.get('intended_goal', intended)
    base_actions, _ = generate_and_parse(intended)
    inj_actions, _ = generate_and_parse(injected)
    decision = policy_decide(injected, intended, inj_actions, sanitizer, semantic, validator, safe_gen, regenerate_fn=generate_and_parse, intended_goal=intended_goal)
    defended = decision['actions']
    env = TextNavigationEnv()
    trace = []
    for a in defended:
        res = env.run_action(a)
        trace.append({'action': a, 'result': res})
    final_state = env.summarize_state()
    ads_val = ads(base_actions, inj_actions)
    svi_val, svi_list = svi(inj_actions)
    gcr_val = gcr(intended_goal, defended, final_state)
    success = attack_success(ads_val, svi_val, gcr_val)
    return {
        'id': entry['id'], 'category': entry['category'],
        'ads': ads_val, 'svi': svi_val, 'gcr': gcr_val, 'attack_success': success,
        'policy_blocked': decision['blocked'], 'reasons': decision['reasons'],
        'san_patterns': decision['san_patterns'], 'suspicion': decision['suspicion'],
        'constrained_regen': decision.get('constrained', False),
        'base_actions': base_actions, 'inj_actions': inj_actions, 'defended': defended,
        'trace': trace, 'final_state': final_state
    }

## 5. Run a Single Sample (change index as needed)

In [ ]:
sample_index = 0  # adjust to explore
result = run_case(attacks[sample_index])
result.keys(), result['ads'], result['gcr'], result['attack_success']

## 6. Inspect Actions & Trace

In [ ]:
import pprint, json
pprint.pprint(result['base_actions'])
pprint.pprint(result['inj_actions'])
pprint.pprint(result['defended'])
print('Trace:')
print(json.dumps(result['trace'], indent=2))
print('Final state:', result['final_state'])
print('Reasons:', result['reasons'])

## 7. Optional: Launch Gradio UI (may not work inside some hosted notebooks without public URL)

In [ ]:
# Optional UI launch
# import gradio as gr
# import threading
# def launch():
#     import gradio_app  # assumes working directory at repo root
# threading.Thread(target=lambda: __import__('gradio_app')).start()
pass